In [ ]:
pip install Flask opencv-python==4.2.0.34
export DISPLAY=:0 && xdpyinfo | grep dimensions
ffmpeg -f x11grab -i :0.0 -framerate 25 -video_size 1024x768 -listen 1 -f mpegts tcp://192.168.56.101:54321 # https://ffmpeg.org/ffmpeg-protocols.html#toc-http
ssh -R 12345:192.168.56.101:54321 iMac

In [ ]:
import socket
import threading 
import cv2 
import time

class VIDEO:          
    def __init__(self):
        self.void_frame = b''  
        self.jpeg_frame = self.void_frame 
        threading.Thread(target=self.opencv , daemon=True).start()    

    def opencv(self):  
        while True:
            cap = cv2.VideoCapture("tcp://localhost:12345")  
            while(cap.isOpened()): 
                ret, frame = cap.read()  
                if not ret: 
                    print('[ Error ] Please check if your source is off~')
                    break   
                ret, jpeg = cv2.imencode('.jpg', frame)
                self.jpeg_frame = jpeg.tobytes()
            cap.release()
            print('[ Warning ] OpenCV lost connection to source!')
            print('[ Warning ] Try reconnection in 3 seconds~')
            time.sleep(3)         

video = VIDEO()  

In [ ]:
import flask  

app = flask.Flask(__name__) 
fps=25
interval = 1/fps
@app.route("/stream.mjpg") 
def mjpg():  
    def generator():      
        while True:     
            time.sleep(interval)  # threading.condition is too shitty according to my test. no condition no lag. 
            frame = video.jpeg_frame 
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

@app.route('/')
def index():
    return flask.redirect(flask.url_for('stream.mjpg'))

app.run('0.0.0.0', 9999)